# DevCollab Analytics & Demo Notebook

**Purpose:** This notebook simulates product metrics, runs analytics, and creates visualizations for the DevCollab project. It's designed as a professional, shareable artifact for teams (Builders Hub, Data Guild, Growth Squad) to inspect KPIs and examples of dashboards.

Sections:

1. Simulated dataset generation
2. KPI calculation (MAU, conversion, retention, revenue)
3. Visualizations (time series, funnel, pie, retention curve)
4. Export examples

_Note: This notebook uses simulated data for demonstration — replace simulation cells with real data sources (Supabase, Prometheus, Grafana) in production._

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

print('Libraries loaded')

## 1) Simulated Dataset Generation
Generating 180 days of simulated metrics: daily active users (DAU), monthly active users (MAU approximation), signups, conversions, revenue, and AI queries.

In [ ]:
# Simulation parameters
np.random.seed(42)
days = 180
start = datetime.today() - timedelta(days=days-1)
dates = [start + timedelta(days=i) for i in range(days)]

# Simulate DAU with growth trend and weekly seasonality
base = np.linspace(500, 8000, days)  # growth from 500 to 8000 DAU over period
seasonality = 500 * np.sin(np.arange(days) * 2 * np.pi / 7)  # weekly pattern
noise = np.random.normal(0, 300, days)
dau = np.maximum(50, (base + seasonality + noise).astype(int))

# Signups proportional to DAU with noise
signups = np.maximum(0, (dau * 0.08 + np.random.normal(0, 5, days)).astype(int))

# Conversions (free -> paid) - funnel rate with small improvements over time
base_conv_rate = 0.02 + np.linspace(0, 0.01, days)  # from 2% to 3%
conversions = (signups * base_conv_rate).astype(int)

# Revenue per conversion (varying)
revenue_per = np.random.normal(25, 5, days)
revenue = (conversions * revenue_per).round(2)

# AI queries per day (grows with DAU)
ai_queries = (dau * (0.5 + np.random.rand(days)*1.5)).astype(int)

# Assemble DataFrame
df = pd.DataFrame({
    'date': dates,
    'dau': dau,
    'signups': signups,
    'conversions': conversions,
    'revenue': revenue,
    'ai_queries': ai_queries
})
df['cumulative_revenue'] = df['revenue'].cumsum()
df['conversion_rate'] = (df['conversions'] / df['signups']).replace([np.inf, -np.inf], 0).fillna(0)
df['ma7_dau'] = df['dau'].rolling(7, min_periods=1).mean()

# show head
df.head()

In [ ]:
# Display sample
import pandas as pd
df.head(10)

## 2) KPI Calculations
Calculate MAU approximation (unique users approximation using DAU aggregate), conversion rates, ARPU, and retention proxy.

In [ ]:
# KPI calculations
# Approximate MAU by taking the max 30-day rolling unique-like proxy via DAU aggregate (simulated proxy)
df['ma30_proxy'] = df['dau'].rolling(30, min_periods=1).mean() * 22  # heuristic proxy for unique users
df['arpu'] = (df['revenue'] / np.maximum(1, df['dau'])).round(4)  # revenue per active user
df['install_to_paid'] = (df['conversions'] / np.maximum(1, df['signups'])).round(4)

kpis = {
    'latest_dau': int(df['dau'].iloc[-1]),
    'avg_dau_30d': float(df['dau'].tail(30).mean()),
    'ma30_proxy': float(df['ma30_proxy'].iloc[-1]),
    'total_revenue': float(df['revenue'].sum()),
    'avg_arpu': float(df['arpu'].mean())
}

kpis, df.tail(3)

## 3) Visualizations
### DAU over time (7-day MA overlay)

In [ ]:
# DAU time series with 7-day moving average
plt.figure(figsize=(12,5))
plt.plot(df['date'], df['dau'])
plt.plot(df['date'], df['ma7_dau'])
plt.title('Daily Active Users (DAU) and 7-day MA')
plt.xlabel('Date')
plt.ylabel('Users')
plt.grid(True)
plt.tight_layout()
plt.show()

### Daily Revenue & Cumulative Revenue

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df['date'], df['revenue'], linestyle='-', marker='o')
plt.title('Daily Revenue')
plt.xlabel('Date')
plt.ylabel('Revenue (USD)')
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,5))
plt.plot(df['date'], df['cumulative_revenue'])
plt.title('Cumulative Revenue')
plt.xlabel('Date')
plt.ylabel('Cumulative Revenue (USD)')
plt.grid(True)
plt.tight_layout()
plt.show()

### Conversion Funnel (Signups → Paid Conversions)

In [ ]:
# Funnel visualization as bar chart for latest 30 days aggregated
agg = df.tail(30).sum()
labels = ['DAU (sum 30d)', 'Signups (30d)', 'Conversions (30d)']
values = [int(df['dau'].tail(30).sum()), int(agg['signups']), int(agg['conversions'])]

plt.figure(figsize=(8,4))
plt.bar(labels, values)
plt.title('30-day Funnel Aggregation')
plt.tight_layout()
plt.show()

### AI Queries Growth

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df['date'], df['ai_queries'])
plt.title('Daily AI Queries')
plt.xlabel('Date')
plt.ylabel('AI Queries')
plt.grid(True)
plt.tight_layout()
plt.show()

### Retention Proxy (Install-to-Paid over time)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(df['date'], df['install_to_paid'])
plt.title('Install-to-Paid (Conversion Rate) Over Time')
plt.xlabel('Date')
plt.ylabel('Conversion Rate')
plt.grid(True)
plt.tight_layout()
plt.show()

## 4) Export Examples
Export sample datasets and KPI summary to CSV for sharing with Growth Squad and Strategy Circle.

In [ ]:
# Export recent 90 days to CSV
recent = df.tail(90).copy()
recent.to_csv('devcollab_recent_90d.csv', index=False)

kpi_df = pd.DataFrame([kpis])
kpi_df.to_csv('devcollab_kpis_summary.csv', index=False)

print('Exported devcollab_recent_90d.csv and devcollab_kpis_summary.csv to workspace /mnt/data')
recent.head()

### Downloadable Exports
The exported CSVs are saved in the notebook workspace. Use the links below to download:

- `devcollab_recent_90d.csv`
- `devcollab_kpis_summary.csv`

## Next Steps & Integration Tips

- Replace simulated data with real sources (Supabase, Prometheus, Grafana). 
- Use Supabase SQL queries or Edge Functions to feed production metrics into this notebook.
- Data Guild can extend the notebook with ML models (churn prediction, segmentation).

**Teams:** Builders Hub (data ingestion), Data Guild (analysis & models), Growth Squad (use reports for campaigns).